In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets, transforms, models
import numpy as np
from tqdm import tqdm 
import pandas as pd
import random
from PIL import Image, ImageOps
from torchvision.transforms import functional as F

In [2]:
# 데이터 경로 설정
train_dir = "/test/final_exam/challenge/train"
augmented_dir = "/home/student/workspace/data"

os.makedirs(augmented_dir, exist_ok=True)


In [20]:
# 증강 함수 정의
def augment_images(class_path, save_path, min_count=300):
    images = [f for f in os.listdir(class_path) if f.endswith(".jpg") or f.endswith(".png")]
    
    # 원본 이미지 불러오기
    image_paths = [os.path.join(class_path, img) for img in images]
    augmented_images = []

    # 좌우 반전
    for img_path in image_paths:
        with Image.open(img_path) as img:
            flipped = ImageOps.mirror(img)
            augmented_images.append(flipped)
            augmented_images.append(img.copy())

    # 랜덤 회전 (시계방향 & 반시계방향 45도 이내)
    final_images = []
    for img in augmented_images:
        for _ in range(2):
            angle = random.uniform(-45, 45)
            rotated = img.rotate(angle)
            final_images.append(rotated)

    # 이미지 저장 (최소 100장 확보)
    while len(final_images) < min_count:
        for img in augmented_images:
            angle = random.uniform(-45, 45)
            rotated = img.rotate(angle)
            final_images.append(rotated)
            if len(final_images) >= min_count:
                break

    # 저장
    os.makedirs(save_path, exist_ok=True)
    for idx, img in enumerate(final_images):
        img.save(os.path.join(save_path, f"aug_{idx}.jpg"))


In [21]:
# 클래스별 증강 실행
for class_name in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        save_path = os.path.join(augmented_dir, class_name)
        augment_images(class_path, save_path)

print("Data augmentation complete.")

Data augmentation complete.
